In [ ]:
import glob
import netCDF4 as nc
from netCDF4 import Dataset
from datetime import datetime
from datetime import date
from dateutil.parser import parse
import numpy as np
from numpy import array
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
## --- Author: Ishita Jalan ---

## Step 1: Converting netcdf to csv

In [ ]:
#function to convert netcdf files to csv (matrix format)

def GLEAMtocsv(dataset,year,dataname):
    os.chdir('D:\TUM\Master_Thesis\Benin\Data\GLEAM\AET_csv')
    latbounds = [6.125,12.500]
    lonbounds = [0.675, 4.300]
    
    ds = nc.Dataset(dataset)
    
    lat = ds.variables['lat'][:] 
    lon = ds.variables['lon'][:]
    aet = ds.variables['E']
    time = ds.variables['time'][:]
    
    # latitude lower and upper index
    latli = np.argmin( np.abs( lat - latbounds[0] ) )
    latui = np.argmin( np.abs( lat - latbounds[1] ) ) 
            
    # longitude lower and upper index
    lonli = np.argmin( np.abs( lon - lonbounds[0] ) )
    lonui = np.argmin( np.abs( lon - lonbounds[1] ) )
    
    startdate = "01-01-"+str(year)
    
    day_counter = 0
    
    for day in range(len(time)):
        enddate = pd.to_datetime(startdate)+ pd.DateOffset(days=day_counter)
        print ("Days Processed", day_counter+1)
        
        TempSubset = ds.variables['E'][day, lonli:lonui, latui:latli]
        temp = np.matrix(TempSubset)
        
        df = pd.DataFrame(data=temp, columns=(lat[latui:latli]), index=(lon[lonli:lonui]))
        day_counter = day_counter+1
  
        # end date timesstamp to string     
        file_date_refined = enddate.strftime('%Y-%m-%d')
        df.to_csv(file_date_refined + '.csv')

In [ ]:
#Calling the function

GLEAMtocsv(r'D:\TUM\Master_Thesis\Benin\Data\GLEAM\E_2005_GLEAM_v35b.nc',2005,'ET_2005')
GLEAMtocsv(r'D:\TUM\Master_Thesis\Benin\Data\GLEAM\E_2006_GLEAM_v35b.nc',2006,'ET_2006')
GLEAMtocsv(r'D:\TUM\Master_Thesis\Benin\Data\GLEAM\E_2007_GLEAM_v35b.nc',2007,'ET_2007')
GLEAMtocsv(r'D:\TUM\Master_Thesis\Benin\Data\GLEAM\E_2008_GLEAM_v35b.nc',2008,'ET_2008')
GLEAMtocsv(r'D:\TUM\Master_Thesis\Benin\Data\GLEAM\E_2009_GLEAM_v35b.nc',2009,'ET_2009')

## Step 2: Compiling into time-series¶

In [ ]:
#compiling daily csv to time series

# Creating a main dataframe to store data from other csv files
def compileAET(filename,directory):
    
    os.chdir(directory)

    df = pd.read_csv(filename)
    row, col = df.shape
    time_series = pd.DataFrame()
    ls =[]

    for r in range(row):
        for c in range(col-1):
            x = df.iloc[r,0]
            y = df.columns.values[c+1]
            col_head = str(y)+","+str(x) #defining the column name using the lat,long
            time_series[col_head] = df.iloc[r,c+1]

    time_series.to_csv("main.csv")

    #Processing data from other csv files to the main csv file
    i=0

    for year in range(2005,2010):
    
        filepath = r'D:\TUM\Master_Thesis\Benin\Data\GLEAM\E_' + str(year) + '_GLEAM_v35b.nc'
        ds = nc.Dataset(filepath)
    
        time = ds.variables['time'][:]
    
        startdate = "01-01-"+str(year)
        day_counter = 0
    
        print("Processing year:", year)
    
        for day in range(len(time)):
            ls=[]
        
            enddate = pd.to_datetime(startdate) + pd.DateOffset(days = day_counter)
            enddate = enddate.strftime('%Y-%m-%d') #removing the time stamp from date
        
            filename = str(enddate)+".csv"
        
            df = pd.read_csv(filename)

            row, col = df.shape
        
            for r in range(row):
            
                for c in range(col-1):
                
                    val = df.iloc[r,c+1]
                    ls.append(val)
            
            time_series.loc[i] = ls
        
            i = i+1 
            day_counter = day_counter+1
        
    time_series.to_csv("AET_2005-09.csv")


In [ ]:
compileAET('2005-01-01.csv','D:\TUM\Master_Thesis\Benin\Data\GLEAM\AET_csv')

## Step 3: Convert to subbasin level data

In [ ]:
# Function to convert time series data for each location point to subbasin level data - averaged for each subbasin

def aettosubbasins(directory,aet,subbasins,dataname):
    
    os.chdir(directory)
    
    subbasins = pd.read_excel(subbasins, sheet_name = 'GLEAM_table', engine = 'openpyxl')
    aet = pd.read_csv(aet)
    
    subbasins.sort_values(by='Subbasin', inplace=True)
    ls_subsn = subbasins['Subbasin'].unique()
    
    final_aet = pd.DataFrame()

    for i in range(len(ls_subsn)):
        
        print("Processing subbasin number:", ls_subsn[i])
        
        sb_df = subbasins[subbasins['Subbasin']==ls_subsn[i]]
        ls_loc = sb_df['Location']
    
        aet_subset = aet[aet['Location'].isin(ls_loc)]
        aet_df = pd.merge(aet_subset, sb_df, on = 'Location')
        aet_df.drop(['OBJECTID', 'Latitude', 'Longitude', 'Subbasin', 'gridcode','Shape_Area'], axis= 1, inplace = True) 
        aet_df = aet_df.iloc[:,1:]
        aet_df.to_csv('aet_subbasin.csv')
        
        sum_area = aet_df['Cell_Area'].sum() #calculating the total area under the current subbasin
        
        temp_xlen = len(aet_df.columns)-1 #determining the number of columns in the dataframe
        
        ls_aet = []
    
        for j in range(temp_xlen):
            aet_df.iloc[:,j] = aet_df.iloc[:,j] * aet_df['Cell_Area']
            avg = aet_df.iloc[:,j].sum()/sum_area #calculating weighted average for each day of the time series
            ls_aet.append(avg)
    
        final_aet[i] = ls_aet
    
    final_aet.to_csv(dataname + '_aet_subbasins.csv')

In [ ]:
aettosubbasins('D:\TUM\Master_Thesis\Benin\Data\GLEAM\GLEAM_AET_processed', 'AET_2005-09.csv',
               'GLEAM_subbasins.xlsx', 'GLEAM_AET')